<a href="https://colab.research.google.com/github/parsafeyz/Recommender_Systems/blob/main/final_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
print('unziping ...')
!unzip -o -j '/content/drive/MyDrive/Colab Notebooks/Recommender Systems/1636213079611301.zip'

unziping ...
Archive:  /content/drive/MyDrive/Colab Notebooks/Recommender Systems/1636213079611301.zip
  inflating: ratings.csv             
  inflating: movies.csv              


In [4]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

اول از همه چندتا کتابخونه ایمپورت شده. pandas رو آوردن که واقعاً تو هر پروژه‌ای با دیتا سروکار داشته باشی، لازم میشه. چون کار با جدول و دیتافریم رو خیلی راحت می‌کنه. با اسم pd ایمپورت شده که توی بقیه‌ی کدها راحت‌تر بشه صداش کرد. بعد از اون sqrt از math ایمپورت شده، فقط همین یکی، احتمالاً چون قراره یه جاهایی فاصله‌ی بین چیزها حساب بشه، مثلاً بین دو نفر یا دو آیتم، و ریشه‌ی دوم لازم میشه. numpy هم اومده که یه پایه‌ی ثابت توی پروژه‌های عددیه. معمولاً وقتی بخوای ماتریس بسازی یا یه عالمه محاسبه انجام بدی، numpy خیلی کمک می‌کنه. آخرش هم یه خط هست که %matplotlib inline نوشته، این فقط مخصوص نوت‌بوک‌هاست، باعث میشه اگه بخوای نمودار رسم کنی، همون‌جا داخل نوت‌بوک نشون بده، نه اینکه یه پنجره‌ی جدا باز کنه.

#Content Based

In [5]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


دیتا ها خوانده شدن نیاز به توضیح نداره

In [6]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '', regex=True)
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


اینجا داره از توی اسم فیلم، سال تولید رو جدا می‌کنه. اول با regex اون (1995) رو می‌کشه بیرون، بعد فقط خود عدد رو درمیاره. بعدش هم اون سال رو از ته عنوان فیلم حذف می‌کنه. آخرشم یه .strip() میزنه که اگه فاصله اضافه مونده، بره پی کارش. آخرش هم head() زده ببینه تمیزکاری درست انجام شده یا نه.



In [7]:
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


تو دیتای خام، ژانرها با | جدا شدن (مثلاً "Action|Adventure|Fantasy"). این خط میاد و با .split('|') هر ژانر رو از هم جدا می‌کنه و می‌ریزه تو لیست. یعنی از یه رشته‌ی طولانی، یه لیست درست می‌کنه که بعداً راحت‌تر بتونیم باهاشون کار کنیم.

In [8]:
moviesWithGenres_df = movies_df.copy()

for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


اول یه کپی از دیتافریم اصلی می‌گیریم به اسم moviesWithGenres_df تا دیتا اصلی خراب نشه.

بعدش می‌ریم رو هر ردیف (یعنی هر فیلم) یه دور می‌زنیم، بعد واسه هر ژانری که اون فیلم داره، توی همون ردیف یه ستون با اسم ژانر می‌سازه و مقدارش رو می‌ذاره ۱.

یعنی اگه یه فیلم “Action” و “Comedy” باشه، توی ستون‌های Action و Comedy عدد ۱ می‌خوره، بقیه ژانرها صفر می‌مونن.

آخرش هم fillna(0) می‌زنیم که هرچی سلول خالی مونده (برای ژانرهایی که اون فیلم نداره)، بشه صفر.

In [9]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [10]:
ratings_df = ratings_df.drop('timestamp', axis=1)
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


اینجا ستون timestamp رو از دیتافریم امتیازها (ratings_df) حذف می‌کنه چون به درد ما نمی‌خوره. یعنی زمان ثبت امتیاز مهم نیست، فقط اینکه کی به چی چند داده برامون مهمه.

axis=1 یعنی داریم یه ستون حذف می‌کنیم، نه یه ردیف.

In [11]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ]
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


یه لیست از دیکشنری ساختیم که تو هر کدوم، اسم فیلم (title) و امتیازی که کاربر داده (rating) هست. مثلاً گفته "Breakfast Club" رو ۵ داده، "Jumanji" رو ۲ و...

بعدش با pd.DataFrame() این لیست رو تبدیل کردیم به یه جدول شیک‌و‌پیک با دو ستون: عنوان فیلم و امتیاز کاربر.

In [12]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop('genres', axis=1).drop('year', axis=1)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


اولش با isin() می‌گردیم ببینیم فیلم‌هایی که کاربر وارد کرده، تو دیتافریم اصلی (movies_df) وجود دارن یا نه. خروجیش inputId هست، یعنی اطلاعات کامل اون فیلم‌ها (مثلاً year و genres و...).

بعد با pd.merge دوتا جدول رو به‌هم می‌چسبونیم: inputMovies که امتیازهای کاربره، و inputId که اطلاعات فیلم‌هاست. اینجوری هم امتیاز داریم، هم متادیتا.

در نهایت ژانر و سال تولید رو پاک می‌کنیم چون تو این مرحله لازم نیستن.

In [13]:
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
257,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
973,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1445,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


اینجا moviesWithGenres_df همون دیتافریمیه که قبلاً درست کردیم و ژانرهاش به صورت باینری بودن (یعنی مثلاً Action = 1 یا 0).

با isin() چک می‌کنیم که movieIdهای داخلش، تو لیست فیلم‌هایی که کاربر دیده (inputMovies) هستن یا نه. اونایی که هستن رو نگه می‌داریم.

در نهایت userMovies میشه یه جدول فقط از فیلم‌هایی که این کاربر امتیاز داده، ولی با تمام اطلاعات ژانری که از قبل آماده کرده بودیم.

In [14]:
userMovies = userMovies.reset_index(drop=True)
userGenreTable = userMovies.drop('movieId', axis=1).drop('title', axis=1).drop('genres', axis=1).drop('year', axis=1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


اول با reset_index(drop=True) شماره‌ی ردیف‌ها رو دوباره مرتب می‌کنیم و ایندکس قبلی رو هم می‌ریزیم دور. این فقط جنبه‌ی مرتب‌سازی داره.

بعدش با drop() چندتا ستون اضافی رو حذف می‌کنیم: movieId، title، genres، و year. چون تو این مرحله فقط به اطلاعات ژانری (همون ستون‌های باینری) نیاز داریم.

نتیجه‌ش میشه یه جدول که هر ردیفش یه فیلمه و هر ستونش یه ژانر. اگه فیلمی تو یه ژانر باشه، اون خونه ۱ میشه، وگرنه ۰.

In [15]:
inputMovies['rating']

,rating
0,3.5
1,2.0
2,5.0
3,4.5
4,5.0


In [16]:
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
userProfile

,0
Adventure,10.0
Animation,8.0
Children,5.5
Comedy,13.5
Fantasy,5.5
Romance,0.0
Drama,10.0
Action,4.5
Crime,5.0
Thriller,5.0


userGenreTable.transpose() یعنی جدول ژانرها رو می‌چرخونیم؛ ژانرها می‌شن ردیف، فیلم‌ها می‌شن ستون.

بعدش .dot(inputMovies['rating']) یعنی ضرب داخلی می‌زنیم بین ژانرها و امتیازهایی که کاربر داده.

چ یه بردار به اسم userProfile که توش نشون می‌ده کاربر چقدر با هر ژانر حال می‌کنه. مثلاً اگه عدد ژانر Action زیاد باشه، یعنی طرف عاشق اکشنه. اگه Comedy صفر باشه، یعنی براش مهم نیست.

In [17]:
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
genreTable = genreTable.drop('movieId', axis=1).drop('title', axis=1).drop('genres', axis=1).drop('year', axis=1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


اولش با .set_index(movieId) میایم movieId رو می‌ذاریم به‌عنوان ایندکس، چون اینطوری راحت‌تر می‌تونیم بعداً به هر فیلم اشاره کنیم.

بعدش مثل قبل، چندتا ستون بی‌ربط رو می‌ریزیم دور: movieId، title، genres، و year، چون تو این مرحله فقط به اون ستون‌های ژانری باینری نیاز داریم. یعنی فقط اینکه هر فیلم تو چه ژانرهایی هست (۰ و ۱).

In [18]:
genreTable.shape

(9742, 20)

In [19]:
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

,0
movieId,
1,0.594406
2,0.293706
3,0.188811
4,0.328671
5,0.188811


genreTable * userProfile: دونه‌به‌دونه ژانرهای هر فیلم رو ضرب می‌کنیم تو علاقه‌ی کاربر به اون ژانر. یعنی مثلاً اگه یه فیلم اکشنه (۱) و کاربر هم خیلی اهل اکشنه (مثلاً نمره ۸)، حاصلش میشه ۸. اگه فیلم تو ژانری باشه که کاربر براش اهمیتی قائل نیست، اون ضرب میشه صفر.

.sum(axis=1): واسه هر فیلم، این مقادیر رو جمع می‌کنیم تا یه نمره‌ی کلی علاقه بسازیم.

/ userProfile.sum(): در نهایت نمره‌ی هر فیلم نرمال‌سازی میشه با مجموع علاقه‌های کاربر به همه ژانرها، که عدد نهایی بین ۰ تا ۱ باشه.

In [20]:
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
recommendationTable_df.head()

,0
movieId,
134853,0.734266
148775,0.685315
6902,0.678322
117646,0.678322
81132,0.671329


توی این خط داریم کل لیست پیشنهادی رو بر اساس بالاترین امتیاز مرتب می‌کنیم. یعنی اون فیلم‌هایی که احتمال داره یوزر بیشتر حال کنه باهاشون، میان اول.

In [21]:
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
559,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
1390,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
2250,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
3460,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
4631,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
5490,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
5819,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
6448,51939,TMNT (Teenage Mutant Ninja Turtles),"[Action, Adventure, Animation, Children, Comed...",2007
6455,52287,Meet the Robinsons,"[Action, Adventure, Animation, Children, Comed...",2007
6462,52462,Aqua Teen Hunger Force Colon Movie Film for Th...,"[Action, Adventure, Animation, Comedy, Fantasy...",2007


بیا از دیتافریم اصلی (movies_df) اون فیلم‌هایی رو پیدا کن که movieIdشون جزو ۲۰تای اول لیست پیشنهادی‌مون باشه.

#Collaborative Filtering

In [22]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

In [23]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '', regex=True)
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

In [24]:
movies_df = movies_df.drop('genres', axis=1)

In [25]:
ratings_df = ratings_df.drop('timestamp', axis=1)

In [26]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ]
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [27]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop('year', axis=1)

In [28]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
0,1,1,4.0
16,1,296,3.0
320,4,296,1.0
422,4,1968,4.0
516,5,1,4.0


In [29]:
userSubsetGroup = userSubset.groupby(['userId'])

In [32]:
userSubsetGroup.get_group(69)

/tmp/ipython-input-2467054568.py:1: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  userSubsetGroup.get_group(69)


,userId,movieId,rating
11622,69,296,5.0


In [42]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

مراحل تا اینجا مشابه بخش اول بود

In [43]:
userSubsetGroup = userSubsetGroup[0:100]

اینجا داریم از یه دیتافریم به اسم userSubsetGroup فقط ۱۰۰ نفر اول یا ۱۰۰ ردیف اول رو نگه می‌داریم.

In [44]:
pearsonCorrelationDict = {}

for name, group in userSubsetGroup:
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    nRatings = len(group)
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

اینجا با یه حلقه for داره رو هر کاربر توی userSubsetGroup می‌چرخه، که اون ۱۰۰ نفر بودن. group.sort_values(by='movieId') و inputMovies.sort_values() باعث میشن ترتیب فیلم‌ها یکی باشه تا امتیازها رو درست با هم مقایسه کنیم. بعد با isin() بررسی می‌کنه کدوم فیلم‌هایی که کاربر ما دیده رو اون یکی کاربر هم دیده. لیست امتیازها با .tolist() درمیاد، بعد هم با چندتا sum() و یه عالمه محاسبه ساده، مقادیر Sxx، Syy و Sxy رو درمیاره که برای فرمول Pearson لازمه. آخرشم با sqrt() و یه if چک می‌کنه اگه همه‌چی اوکیه، شباهت رو حساب کنه. خروجی میره تو pearsonCorrelationDict که یه دیکشنریه با اسم کاربرا و میزان شباهتشون به ما.

In [45]:
pearsonCorrelationDict

{(91,): 0.43852900965351443,
 (177,): 0.0,
 (219,): 0.45124262819713973,
 (274,): 0.716114874039432,
 (298,): 0.9592712306918567,
 (414,): 0.9376144618769914,
 (474,): 0.11720180773462392,
 (477,): 0.4385290096535153,
 (480,): 0.7844645405527362,
 (483,): 0.08006407690254357,
 (599,): 0.7666866491579839,
 (608,): 0.920736884379251,
 (50,): 0.15713484026367722,
 (57,): -0.7385489458759964,
 (68,): 0.0,
 (103,): 0.5222329678670935,
 (135,): 0.8703882797784892,
 (182,): 0.9428090415820635,
 (202,): 0.5222329678670935,
 (217,): 0.30151134457776363,
 (226,): 0.9438798074485389,
 (288,): 0.6005325641789633,
 (307,): 0.9655810287305759,
 (318,): 0.44486512077567225,
 (322,): 0.5057805388588731,
 (330,): 0.9035942578600878,
 (357,): 0.5606119105813882,
 (434,): 0.9864036607532465,
 (448,): 0.30151134457776363,
 (469,): 0.8164965809277261,
 (561,): 0.5222329678670935,
 (600,): 0.18442777839082938,
 (606,): 0.9146591207600472,
 (610,): -0.47140452079103173,
 (18,): 1.0,
 (19,): -0.5,
 (21,): 0,


In [46]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.438529,"(91,)"
1,0.000000,"(177,)"
2,0.451243,"(219,)"
3,0.716115,"(274,)"
4,0.959271,"(298,)"


با pd.DataFrame.from_dict() کل pearsonCorrelationDict رو می‌ریزه توی یه جدول. چون کلیدهاش اسم کاربرا یا همون userId بودن، orient='index' می‌گه اونا بشن سطرهای دیتافریم. بعد با pearsonDF.columns = ['similarityIndex'] یه اسم مرتب براش می‌ذاره. بعد userId رو دوباره به‌صورت یه ستون جداگانه اضافه می‌کنه، چون الان توی index بودن. در نهایت هم با index = range(...) ایندکس رو ریسِت می‌کنه تا یه جدول صاف و صوف داشته باشیم.

In [47]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
43,1.0,"(132,)"
34,1.0,"(18,)"
46,1.0,"(144,)"
63,1.0,"(305,)"
82,1.0,"(489,)"


اینجا اومده از بین همه‌ی کاربرا، اونایی که بیشترین شباهت رو به کاربر ما داشتن انتخاب کرده. با sort_values() بر اساس similarityIndex مرتبشون کرده (از بیشترین به کمترین چون ascending=False) و بعدش با [0:50] فقط ۵۰تای اول رو نگه داشته

In [48]:
topUsers['userId'] = topUsers['userId'].apply(lambda x: x[0] if isinstance(x, tuple) else x)
topUsers['userId'] = topUsers['userId'].astype(int)
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,1.0,132,1,2.0
1,1.0,132,17,3.0
2,1.0,132,29,2.0
3,1.0,132,32,3.0
4,1.0,132,34,1.5


اینجا داره یه تمیزکاری کوچیک ولی مهم انجام میده. بعضی از userIdها انگار به صورت tuple ذخیره شدن (مثلاً (12,) به جای 12). با اون apply() و lambda میاد چک می‌کنه اگه tuple بودن، فقط عنصر اولش رو برداره. بعد با .astype(int) مطمئن میشه که همه‌شون عدد صحیح باشن.

In [49]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,132,1,2.0,2.0
1,1.0,132,17,3.0,3.0
2,1.0,132,29,2.0,2.0
3,1.0,132,32,3.0,3.0
4,1.0,132,34,1.5,1.5


اینجا اومده یه ستون جدید ساخته به اسم weightedRating، یعنی «امتیاز وزنی». داره امتیازی که هر کاربر به یه فیلم داده رو ضرب می‌کنه تو شباهتش به کاربر ما (similarityIndex * rating). این‌جوری کسی که خیلی شبیه ماست، نظرش ارزش بیشتری داره. مثلاً اگه یه فیلمو یه آدم کاملاً هم‌سلیقه بهمون ۵ داده، اون بیشتر به چشم میاد تا وقتی یه آدم غیرمرتبط همون فیلمو ۵ داده باشه.

In [50]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,36.354096,133.167946
2,31.005292,94.904257
3,8.783859,26.381456
4,0.866025,1.732051
5,7.165336,19.775255


با .groupby('movieId') میاد کل امتیازای وزنی رو برای هر فیلم جمع می‌زنه. یعنی می‌گه "هر فیلم، در کل از همه‌ی آدمای شبیه ما، چه میزان امتیاز وزنی گرفته؟"
فقط دوتا ستون نگه می‌داره: یکی similarityIndexها (که می‌شن وزن‌ها)، یکی هم weightedRatingها (که می‌شن امتیازهای ضرب‌شده).
بعد اسم ستون‌ها رو با columns = ... عوض می‌کنه تا واضح‌تر باشن.

In [51]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.663080,1
2,3.060905,2
3,3.003402,3
4,2.000000,4
5,2.759850,5


با این خط، داریم نمره نهایی هر فیلم رو حساب می‌کنیم: امتیازهای وزنی تقسیم بر مجموع شباهت‌ها. یعنی weighted average recommendation score دقیقاً نشون می‌ده که «اگه بخوایم نظر اونایی که شبیه ماییم رو با هم ترکیب کنیم، این فیلم چه نمره‌ای می‌گیره؟

In [52]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
905,5.0,905
7579,5.0,7579
3310,5.0,3310
1211,5.0,1211
167064,5.0,167064
187593,5.0,187593
3951,5.0,3951
158027,5.0,158027
160644,5.0,160644


تو این خط اومده لیست نهایی فیلم‌ها رو بر اساس نمره‌ی پیشنهادی مرتب کرده، از بیشترین به کمترین. یعنی اون فیلمایی که احتمال داره بیشترین حالو به کاربر بده، می‌رن بالا. بعد با head(10)، فقط ۱۰تای اول رو نشون می‌ده

In [53]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
687,905,It Happened One Night,1934
912,1211,"Wings of Desire (Himmel über Berlin, Der)",1987
2484,3310,"Kid, The",1921
2947,3951,Two Family House,2000
4969,7579,Pride and Prejudice,1940
9284,158027,SORI: Voice from the Heart,2016
9337,160644,Indignation,2016
9367,162414,Moonlight,NaN
9443,167064,I Am Not Your Negro,2017
9709,187593,Deadpool 2,2018


اینجا recommendation_df.head(10)['movieId'].tolist() میاد آیدی اون ۱۰ فیلم تاپ رو در میاره، بعد با isin() تو movies_df می‌گرده دنبال مشخصاتشون.
نتیجه‌ش میشه یه جدول خوشگل شامل اسم، ژانر و سال ساخت ۱۰ فیلمی که الگوریتم فکر می‌کنه خیلی با سلیقه‌ی کاربر جور درمیاد.